In [3]:
import json
from PIL import Image, ImageDraw, ImageFont

def draw_dungeon(json_file, output_file='dungeon_map.png', cell_size=20):
    # Carregar o JSON
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        
    cells = data["cells"]
    cell_bit = data["cell_bit"]
    rooms = data["rooms"]
    
    rows = len(cells)
    cols = len(cells[0]) if rows > 0 else 0

    img = Image.new("RGB", (cols * cell_size, rows * cell_size), "white")
    draw = ImageDraw.Draw(img)

    # Fonte para texto (opcional, se não tiver fonte pode tentar sem)
    # Ajuste o caminho da fonte conforme necessário
    try:
        font = ImageFont.truetype("arial.ttf", 14)
    except:
        font = ImageFont.load_default()

    # Desenhar o piso e as áreas
    for r in range(rows):
        for c in range(cols):
            val = cells[r][c]
            x1 = c * cell_size
            y1 = r * cell_size
            x2 = x1 + cell_size
            y2 = y1 + cell_size
            
            # Sala ou corredor: piso branco
            if (val & cell_bit["room"]) or (val & cell_bit["corridor"]):
                cell_color = (255, 255, 255)
            else:
                cell_color = (230, 230, 230)
            
            draw.rectangle([x1, y1, x2, y2], fill=cell_color)

            # Desenhar perímetro ou paredes
            if (val & cell_bit["perimeter"]):
                draw.line([x1, y1, x2, y1], fill="black", width=2)
                draw.line([x1, y2, x2, y2], fill="black", width=2)
                draw.line([x1, y1, x1, y2], fill="black", width=2)
                draw.line([x2, y1, x2, y2], fill="black", width=2)

    # Desenhar número das salas
    # Os dados de sala incluem col/row/west/east/north/south.
    # Assumiremos que (west,east) e (north,south) dão a bounding box em "cells".
    # Tomar o centro e desenhar o número.
    for i, room in enumerate(rooms):
        if room is None:
            continue
        # Calcular centro da sala
        # Usar a média das coordenadas "west,east" e "north,south"
        # Estas coord. parecem ser em células, então podemos usar diretamente.
        cx = (room["west"] + room["east"]) // 2
        cy = (room["north"] + room["south"]) // 2
        x_text = cx * cell_size + cell_size // 4
        y_text = cy * cell_size + cell_size // 4
        draw.text((x_text, y_text), str(room["id"]), fill="red", font=font)

    # Desenhar portas
    # Portas estão dentro de cada room em "doors"
    # Cada porta tem col/row e type. Podemos desenhar um símbolo. 
    # Por exemplo, "D" para porta normal, "S" para secreta, "T" para armadilhada.
    for room in rooms:
        if room is None or "doors" not in room:
            continue
        for direction, door_list in room["doors"].items():
            for door in door_list:
                dcol = door["col"]
                drow = door["row"]
                # Escolher símbolo dependendo do tipo da porta
                dtype = door["type"]
                if "secret" in dtype:
                    symbol = "S"
                elif "trapped" in dtype:
                    symbol = "T"
                elif "locked" in dtype:
                    symbol = "L"
                else:
                    symbol = "D"
                
                x_text = dcol * cell_size + cell_size // 4
                y_text = drow * cell_size + cell_size // 4
                draw.text((x_text, y_text), symbol, fill="blue", font=font)

    img.save(output_file)
    print(f"Mapa salvo em {output_file}")

In [4]:
# Exemplo de uso:
draw_dungeon('d1.json', 'dungeon_map.png')


Mapa salvo em dungeon_map.png
